## Bronze Layer

#### Criei um volume delta já que no free tier o acesso a mnt/ é negado

#### Verificando os dados de teste disponíveis

In [0]:
%py
# Definir caminhos base
# BRONZE_PATH = "/mnt/delta/bronze"
BRONZE_PATH = "/Volumes/workspace/default/delta/bronze/retail"
SILVER_PATH = "/Volumes/workspace/default/delta/silver/retail"


# Criando diretório Silver
# Definir caminhos base


In [0]:
display(dbutils.fs.ls(BRONZE_PATH))


### Dataset retail silver


In [0]:
silver_map = {
    "tmp_customers": f"{BRONZE_PATH}/customers",
    "tmp_company_employees": f"{BRONZE_PATH}/company_employees",
    "tmp_active_promotions": f"{BRONZE_PATH}/active_promotions",
    "tmp_loyalty_segment": f"{BRONZE_PATH}/loyalty_segment",
    "tmp_products": f"{BRONZE_PATH}/products",
    "tmp_promotions": f"{BRONZE_PATH}/promotions",
    "tmp_purchase_orders": f"{BRONZE_PATH}/purchase_orders",
    "tmp_sales_orders": f"{BRONZE_PATH}/sales_orders",
    "tmp_sales_stream": f"{BRONZE_PATH}/sales_stream",
    "tmp_suppliers": f"{BRONZE_PATH}/suppliers"
}

for view_name, path in silver_map.items():
    print(f"Loading {view_name} from {path}")
    spark.read.format("delta").load(path).createOrReplaceTempView(view_name)
# DBTITLE 1,customers

### Lendo as tabelas

In [0]:
%sql
SELECT * FROM tmp_suppliers 

In [0]:
from pyspark.sql import functions as F

df_exploded = (
    _sqldf 
    # transforma string em array usando a vírgula como separador
    .withColumn("item_array", F.split(F.col("items_provided"), ","))
    # explode a coluna array em múltiplas linhas
    .withColumn("item", F.explode(F.col("item_array")))
    # opcional: remover espaços extras
    .withColumn("item", F.trim(F.col("item")))
    # remover coluna temporária
    .drop("item_array", "items_provided", "street", "number", "unit", "region", "district", "postcode", "city", "state", "lat", "lon", "street_ship", "number_ship", "unit_ship", "region_ship", "district_ship", "postcode_ship", "city_ship", "state_ship", "lat_ship", "lon_ship")
)

display(df_exploded)

In [0]:
%sql
SELECT
    o.SUPPLIER_ID,
    TRIM(i) AS item
FROM tmp_suppliers o
LATERAL VIEW EXPLODE(SPLIT(o.items_provided, ',')) AS i


In [0]:
%sql
CREATE OR REPLACE TABLE workspace.retail.silver_purchase_order
USING DELTA
AS
SELECT * FROM tmp_purchase_orders